## 4SQ AND GEOJSON

In [3]:
from pymongo import MongoClient
import pandas as pd
import json
import geojson
import os
import requests
from dotenv import load_dotenv
import time
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [4]:
load_dotenv()

True

In [5]:
client = MongoClient("localhost:27017")
db = client["cities_project"]

**GEO JSON TO GET THE CITIES LOCATION AND NAIGHBOURHOODS, AND 4SQR QUERIES**

**GEO JSON**

    SAN FRANCISCO MONGO IMPORT 

In [6]:
def load_geo_json(path):

    ''' Function to load geo json.'''

    with open(path) as f:
        geo_json = json.load(f)
    return geo_json

In [7]:
san_francisco = load_geo_json("../data/SanFrancisco.Neighborhoods.json")
san_francisco["features"][0]

{'geometry': {'geometries': [{'type': 'Polygon',
    'coordinates': [[[-122.391701, 37.794113],
      [-122.39198, 37.793906],
      [-122.391614, 37.793571],
      [-122.391714, 37.793459],
      [-122.388816, 37.791005],
      [-122.388932, 37.790919],
      [-122.388616, 37.790348],
      [-122.388076, 37.790518],
      [-122.388375, 37.790334],
      [-122.388225, 37.790032],
      [-122.385852, 37.790951],
      [-122.385496, 37.790559],
      [-122.387589, 37.789838],
      [-122.387289, 37.789347],
      [-122.385303, 37.789838],
      [-122.38512, 37.789313],
      [-122.387174, 37.788807],
      [-122.387032, 37.788255],
      [-122.385261, 37.788537],
      [-122.385136, 37.788156],
      [-122.38739, 37.787736],
      [-122.387415, 37.787269],
      [-122.3845, 37.787437],
      [-122.384342, 37.785728],
      [-122.387577, 37.785485],
      [-122.38765, 37.784929],
      [-122.385546, 37.785009],
      [-122.385455, 37.784711],
      [-122.387477, 37.784493],
      [-122.38

In [8]:
def upload_geo_mongo(geojson, name_col):
    
    ''' Function to upload geo json to mongo.'''

    c = db.get_collection(name_col)

    for i in geojson["features"]:
        c.insert_one({"geo": i["geometry"]["geometries"][0], "name": i["properties"]})

In [9]:
upload_geo_mongo(san_francisco, "SanFranciscoNeigh")

    NEW YORK MONGO IMPORT 

In [10]:
new_york = load_geo_json("../data/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson")
new_york["features"][0]

{'type': 'Feature',
 'properties': {'neighborhood': 'Allerton',
  'boroughCode': '2',
  'borough': 'Bronx',
  '@id': 'http://nyc.pediacities.com/Resource/Neighborhood/Allerton'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-73.84859700000018, 40.871670000000115],
    [-73.84582253683678, 40.870239076236174],
    [-73.85455918463374, 40.85995383576425],
    [-73.85466543306826, 40.859585694988056],
    [-73.85638870335896, 40.85759363530448],
    [-73.86888180915341, 40.857223150158326],
    [-73.86831755272824, 40.85786206225831],
    [-73.86955371467232, 40.85778409560018],
    [-73.87102485762065, 40.857309948816905],
    [-73.87048054998716, 40.865413584098484],
    [-73.87055489856489, 40.86970279858986],
    [-73.86721594442561, 40.86968966363671],
    [-73.85745, 40.86953300000018],
    [-73.85555000000011, 40.871813000000145],
    [-73.85359796757658, 40.8732883686742],
    [-73.84859700000018, 40.871670000000115]]]}}

In [11]:
def upload_geo_mongo_variable(geojson, name_col):
    
    ''' Function to upload geo json to mongo.'''

    c = db.get_collection(name_col)

    for i in geojson["features"]:
        c.insert_one({"geo": i["geometry"], "name": i["properties"]})

In [12]:
upload_geo_mongo_variable(new_york, "NewYorkNeigh")

    SEATTLE MONGO IMPORT 

In [13]:
seattle = load_geo_json("../data/Neighborhood_Map_Atlas_Neighborhoods.geojson")
seattle["features"][0]

{'type': 'Feature',
 'properties': {'OBJECTID': 27,
  'L_HOOD': 'Ballard',
  'S_HOOD': 'Loyal Heights',
  'S_HOOD_ALT_NAMES': None,
  'Shape__Area': 21320655.5455933,
  'Shape__Length': 18831.00959637},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-122.376336564723, 47.6759176989664],
    [-122.376707907517, 47.6759982290459],
    [-122.377903057631, 47.6759978849021],
    [-122.379988625303, 47.6759893429385],
    [-122.38182788443, 47.675992079457],
    [-122.383602752477, 47.675979810424],
    [-122.385435721425, 47.6759587956618],
    [-122.387087434206, 47.6759521343769],
    [-122.387702391216, 47.6759476875929],
    [-122.388955759168, 47.6759424488678],
    [-122.390449346263, 47.6759259868868],
    [-122.391773109273, 47.6759158131504],
    [-122.392827172066, 47.6759153143353],
    [-122.392956550545, 47.6759141859541],
    [-122.392956303464, 47.6763287099834],
    [-122.392959618409, 47.676929558447],
    [-122.392961263359, 47.6775085790216],
    [-122.392971502986

In [14]:
upload_geo_mongo_variable(seattle, "SeattleNeigh")

**4SQ**

COORDINATES FOR EACH CITY

In [15]:
token = os.getenv("token")

    SAN FRANCISCO

In [16]:
def get_coordinates(where):

    ''' Function to get coordinates of cities.'''

    url_geocode = f"https://geocode.xyz/{where}?json=1"
    res = requests.get(url_geocode)
    san_francisco_loc = res.json()
    lat = san_francisco_loc["latt"]
    lon = san_francisco_loc["longt"]
    coordinates = lat+","+lon
    return coordinates

In [17]:
san_francisco_loc = get_coordinates("501 Twin Peaks Blvd, San Francisco, CA 94114, Estados Unidos")
san_francisco_loc

'37.75404,-122.44721'

    NEW YORK

In [18]:
new_york_loc = get_coordinates("Central Park N, New York, New York")
new_york_loc

'40.79724,-73.94968'

    SEATTLE

In [19]:
seattle_loc = get_coordinates("1027 S Washington St, Seattle, WA 98104, EE. UU.")
seattle_loc

'47.60065,-122.31819'

FORSQUARE QUERY

In [20]:
def upload_category_mongo(json, name_cat):
    
    ''' Function to upload category json into mongo.
    It takes 2 args:
        - json
        - name of the category.'''

    c = db.get_collection(name_cat)

    for i in json:
        c.insert_one(i)

In [21]:
def requests_for_foursquare (venue, lat, lon, radius, sort = "DISTANCE" ,limit = 50):

    ''' Function to get places I'm looking for in a specific location'''

    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{lon}&radius={radius}&sort={sort}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("te has pasado")

In [22]:
'''def foursquare_p (list_,coordinates):
    
    Function to create dictionaries with only the needed information.

    lat, lon = str(coordinates).split(',')

    categ = ["SFstarbucks", "SFbaskStadium", "SFschools"]
    name_cat = [i for i in categ]

    for item in list_:

        res = requests_for_foursquare (lat, lon, radius=100000, categ =item, sort = "DISTANCE" ,limit = 1)
        print(res)
        venue = res["results"]  
        results = []

        for i in venue:
            if "Starbucks" in i["name"]:
                name = i["name"]
                lon = i["geocodes"]["main"]["longitude"]
                lat = i["geocodes"]["main"]["latitude"]

                results.append( {
                    "name": name,
                    "lat": lat,
                    "lon": lon
                    })
            else:
                name = i["name"]
                lon = i["geocodes"]["main"]["longitude"]
                lat = i["geocodes"]["main"]["latitude"]

                results.append( {
                    "name": name,
                    "lat": lat,
                    "lon": lon
                    })

        upload_category_mongo(results, name_cat)

        return results'''

'def foursquare_p (list_,coordinates):\n    \n    Function to create dictionaries with only the needed information.\n\n    lat, lon = str(coordinates).split(\',\')\n\n    categ = ["SFstarbucks", "SFbaskStadium", "SFschools"]\n    name_cat = [i for i in categ]\n\n    for item in list_:\n\n        res = requests_for_foursquare (lat, lon, radius=100000, categ =item, sort = "DISTANCE" ,limit = 1)\n        print(res)\n        venue = res["results"]  \n        results = []\n\n        for i in venue:\n            if "Starbucks" in i["name"]:\n                name = i["name"]\n                lon = i["geocodes"]["main"]["longitude"]\n                lat = i["geocodes"]["main"]["latitude"]\n\n                results.append( {\n                    "name": name,\n                    "lat": lat,\n                    "lon": lon\n                    })\n            else:\n                name = i["name"]\n                lon = i["geocodes"]["main"]["longitude"]\n                lat = i["geocodes"]["

In here I tried to do a for loop to get an duploadall the collections at once but didn't work, so I just put insted of a list, a category.

In [104]:
def foursquare_places (venue,coordinates,name_cat):
    
    ''' Function to create dictionaries with only the needed information.'''

    lat, lon = str(coordinates).split(',')

    res = requests_for_foursquare (venue, lat, lon, radius=50000, sort = "DISTANCE" ,limit = 50)
    venue = res["results"]
    print(venue)
    results = []

    for i in venue:
        name = i["name"]
        lon = i["geocodes"]["main"]["longitude"]
        lat = i["geocodes"]["main"]["latitude"]

        results.append( {
            "name": name,
            "lat": lat,
            "lon": lon
            })
    
    upload_category_mongo(results, name_cat)

    return results

In [24]:
'''def foursquare_starbucks (venue,coordinates,name_cat):
    
    Function to create dictionaries for the starbucks category.

    lat, lon = str(coordinates).split(',')

    res = requests_for_foursquare (venue,lat, lon, radius=70000, sort = "POPULARITY" ,limit = 50)
    venue = res["results"]  
    results = []

    for i in venue:
        if "Starbucks" or "starbucks" in i["name"]:
            name = i["name"]
            lon = i["geocodes"]["main"]["longitude"]
            lat = i["geocodes"]["main"]["latitude"]

            results.append( {
                "name": name,
                "lat": lat,
                "lon": lon
                })
            
    upload_category_mongo(results, name_cat)

    return results'''

'def foursquare_starbucks (venue,coordinates,name_cat):\n    \n    Function to create dictionaries for the starbucks category.\n\n    lat, lon = str(coordinates).split(\',\')\n\n    res = requests_for_foursquare (venue,lat, lon, radius=70000, sort = "POPULARITY" ,limit = 50)\n    venue = res["results"]  \n    results = []\n\n    for i in venue:\n        if "Starbucks" or "starbucks" in i["name"]:\n            name = i["name"]\n            lon = i["geocodes"]["main"]["longitude"]\n            lat = i["geocodes"]["main"]["latitude"]\n\n            results.append( {\n                "name": name,\n                "lat": lat,\n                "lon": lon\n                })\n            \n    upload_category_mongo(results, name_cat)\n\n    return results'

FINDING REQUIREMENTS IN EACH CITY

    SAN FRANCISCO

In [25]:
# fer una llista amb totes les coses a buscar i passarli per cada for item in list la funcio i despres a dins del for loop igualar el resultat de la funcio al nom de la llista. restaurants i funcio.

basketball_stadiums = "10061,18008"
schools = "12056,12057,12058,12059,12060"
caffe = "13032,13034,13035,13037"
airport = "19031,19040"
train_stations = "19066,19050,19063,19070,19047"
pet_grooming = "11134"
dog_parks = "16033"
bars_clubs = "13003,13009,13019,10032"
vegan_vegetarian = "13377"

#categories = ["10061,18008","12056,12057,12058,12059,12060","13032,13034,13035,13037","19031,19040","19066,19050,19063,19070,19047","11134","16033","13003,13009,13019,10032","13377"]

#I tried to use the categories list for the function and didn't work so I just decided that It would be easier to get a result for each category.


In [54]:
starbucks_SF = foursquare_places ("Starbucks",san_francisco_loc, "SFstarbucks")

[{'fsq_id': '4310fe80f964a52051271fe3', 'categories': [{'id': 13035, 'name': 'Coffee Shop', 'short_name': 'Coffee Shop', 'plural_name': 'Coffee Shops', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}}], 'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da', 'name': 'Starbucks'}], 'closed_bucket': 'VeryLikelyOpen', 'distance': 1139, 'geocodes': {'main': {'latitude': 37.744586, 'longitude': -122.452602}, 'roof': {'latitude': 37.744586, 'longitude': -122.452602}}, 'link': '/v3/places/4310fe80f964a52051271fe3', 'location': {'address': '675 Portola Dr', 'census_block': '060750306002002', 'country': 'US', 'cross_street': 'btwn Teresita Blvd & Fowler Ave', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '675 Portola Dr (btwn Teresita Blvd & Fowler Ave), San Francisco, CA 94127', 'locality': 'San Francisco', 'postcode': '94127', 'region': 'CA'}, 'name': 'Starbucks', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_

In [55]:
basketball_stadiums_SF = foursquare_places ("Basketball stadium",san_francisco_loc, "SFbaskStadium")

[{'fsq_id': '4b805573f964a520026930e3', 'categories': [{'id': 10051, 'name': 'Stadium', 'short_name': 'Stadium', 'plural_name': 'Stadiums', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 1600, 'geocodes': {'main': {'latitude': 37.767456, 'longitude': -122.453888}, 'roof': {'latitude': 37.767456, 'longitude': -122.453888}}, 'link': '/v3/places/4b805573f964a520026930e3', 'location': {'address': '755 Stanyan St', 'census_block': '060759803001034', 'country': 'US', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '755 Stanyan St, San Francisco, CA 94117', 'locality': 'San Francisco', 'postcode': '94117', 'region': 'CA'}, 'name': 'Kezar Pavilion', 'related_places': {'parent': {'fsq_id': '445e36bff964a520fb321fe3', 'categories': [{'id': 16039, 'name': 'Urban Park', 'short_name': 'Urban Park', 'plural_name': 'Urban Parks', 'icon': {'prefix': 'https://ss3.4sq

In [56]:
schools_SF = foursquare_places ("schools", san_francisco_loc, "SFschools")

[{'fsq_id': '5311327c498e2ec9462d7d93', 'categories': [{'id': 12061, 'name': 'Private School', 'short_name': 'Private School', 'plural_name': 'Private Schools', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 2071, 'geocodes': {'main': {'latitude': 37.771748, 'longitude': -122.439668}, 'roof': {'latitude': 37.771748, 'longitude': -122.439668}}, 'link': '/v3/places/5311327c498e2ec9462d7d93', 'location': {'address': '1155 Page St', 'census_block': '060750167004003', 'country': 'US', 'cross_street': 'Baker St', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '1155 Page St (Baker St), San Francisco, CA 94117', 'locality': 'San Francisco', 'postcode': '94117', 'region': 'CA'}, 'name': 'French American International School', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4b5f4c02f964a52098b229e3', 'categories': [{'id': 12014, 'name': 'College 

In [57]:
airport_SF = foursquare_places ("airport",san_francisco_loc,"SFairport")

[{'fsq_id': '4f7ba94ce4b06978a72ee52c', 'categories': [{'id': 13006, 'name': 'Beer Bar', 'short_name': 'Beer Bar', 'plural_name': 'Beer Bars', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}, {'id': 13029, 'name': 'Brewery', 'short_name': 'Brewery', 'plural_name': 'Breweries', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_', 'suffix': '.png'}}, {'id': 13297, 'name': 'Latin American Restaurant', 'short_name': 'Latin American', 'plural_name': 'Latin American Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/latinamerican_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'Unsure', 'distance': 1825, 'geocodes': {'main': {'latitude': 37.761097, 'longitude': -122.428574}, 'roof': {'latitude': 37.761097, 'longitude': -122.428574}}, 'link': '/v3/places/4f7ba94ce4b06978a72ee52c', 'location': {'address': '3801 18th St', 'census_block': '060750206011002', 'country': 'US', 'cross_street': 'at 

In [58]:
train_stations_SF = foursquare_places ("train station",san_francisco_loc, "SFtrainStations")

[{'fsq_id': 'd696604e8f9c40ab29ed9085', 'categories': [{'id': 19047, 'name': 'Rail Station', 'short_name': 'Rail Station', 'plural_name': 'Rail Stations', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'Unsure', 'distance': 1245, 'geocodes': {'main': {'latitude': 37.748035, 'longitude': -122.459183}, 'roof': {'latitude': 37.748035, 'longitude': -122.459183}}, 'link': '/v3/places/d696604e8f9c40ab29ed9085', 'location': {'address': 'Laguna Honda Blvd', 'census_block': '060750304001009', 'country': 'US', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': 'Laguna Honda Blvd, San Francisco, CA 94116', 'locality': 'San Francisco', 'postcode': '94116', 'region': 'CA'}, 'name': 'Forest Hill Station', 'related_places': {'children': [{'fsq_id': '6af0c47e9c644a193c033da2', 'categories': [{'id': 19054, 'name': 'Public Transportation', 'short_name': 'Public Transportation', 'plural_name': 'Public Transp

In [59]:
pet_grooming_SF = foursquare_places ("pet grooming", san_francisco_loc,"SFpet_grooming")

[{'fsq_id': '4b071dabf964a5208df722e3', 'categories': [{'id': 11134, 'name': 'Pet Grooming Service', 'short_name': 'Pet Grooming Service', 'plural_name': 'Pet Grooming Services', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/dogrun_', 'suffix': '.png'}}, {'id': 17110, 'name': 'Pet Supplies Store', 'short_name': 'Pet Supplies Store', 'plural_name': 'Pet Supplies Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 858, 'geocodes': {'main': {'latitude': 37.750878, 'longitude': -122.438288}, 'roof': {'latitude': 37.750878, 'longitude': -122.438288}}, 'link': '/v3/places/4b071dabf964a5208df722e3', 'location': {'address': '4299 24th St', 'census_block': '060750213001001', 'country': 'US', 'cross_street': 'btwn Diamond & Douglass', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '4299 24th St (btwn Diamond & Douglass), San Francisco, CA 941

In [60]:
dog_parks_SF = foursquare_places ("dog park", san_francisco_loc,"SFdog_parks")

[{'fsq_id': '4f73a5e2e4b0ae979e5d0e1e', 'categories': [{'id': 16000, 'name': 'Landmarks and Outdoors', 'short_name': 'Landmarks and Outdoors', 'plural_name': 'Landmarks and Outdoors', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/default_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 681, 'geocodes': {'main': {'latitude': 37.751568, 'longitude': -122.454449}, 'roof': {'latitude': 37.751568, 'longitude': -122.454449}}, 'link': '/v3/places/4f73a5e2e4b0ae979e5d0e1e', 'location': {'census_block': '060750305003000', 'country': 'US', 'cross_street': '', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': 'San Francisco, CA 94131', 'locality': 'San Francisco', 'postcode': '94131', 'region': 'CA'}, 'name': 'Sutro Reservoir', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4a9de718f964a520ce3920e3', 'categories': [{'id': 16032, 'name': 'Park', 'short_name': 'Park', 'plural_name': 'Parks', 'icon': {

In [105]:
bars_clubs_SF = foursquare_places ("night club",san_francisco_loc,"SFbars_clubs")

[{'fsq_id': '4abc421df964a520ef8620e3', 'categories': [{'id': 10037, 'name': 'Concert Hall', 'short_name': 'Concert Hall', 'plural_name': 'Concert Halls', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_', 'suffix': '.png'}}, {'id': 10039, 'name': 'Music Venue', 'short_name': 'Music Venue', 'plural_name': 'Music Venues', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_', 'suffix': '.png'}}, {'id': 10043, 'name': 'Theater', 'short_name': 'Theater', 'plural_name': 'Theaters', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 16629, 'geocodes': {'main': {'latitude': 37.80807, 'longitude': -122.270584}, 'roof': {'latitude': 37.808105, 'longitude': -122.270541}}, 'link': '/v3/places/4abc421df964a520ef8620e3', 'location': {'address': '1807 Telegraph Ave', 'census_block': '06001

In [62]:
vegan_vegetarian_SF = foursquare_places ("vegan restaurant", san_francisco_loc,"SFvegan_vegetarian")

[{'fsq_id': '4eac41a5dab40d132703fc44', 'categories': [{'id': 13046, 'name': 'Ice Cream Parlor', 'short_name': 'Ice Cream', 'plural_name': 'Ice Cream Parlors', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_', 'suffix': '.png'}}, {'id': 13049, 'name': 'Diner', 'short_name': 'Diner', 'plural_name': 'Diners', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_', 'suffix': '.png'}}, {'id': 13334, 'name': 'Sandwich Spot', 'short_name': 'Sandwich Spot', 'plural_name': 'Sandwich Spots', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 1394, 'geocodes': {'main': {'latitude': 37.766429, 'longitude': -122.450244}, 'roof': {'latitude': 37.766429, 'longitude': -122.450244}}, 'link': '/v3/places/4eac41a5dab40d132703fc44', 'location': {'address': '815 Cole St', 'address_extended': '# 3907', 'census_block': '060750171023006', 'country': 'US', 'cross_street

    NEW YORK

In [82]:
starbucks_NYC = foursquare_places ("Starbucks",new_york_loc,"NYCstarbucks")

[{'fsq_id': '57fd4579498e7ebff233ef5a', 'categories': [{'id': 13035, 'name': 'Coffee Shop', 'short_name': 'Coffee Shop', 'plural_name': 'Coffee Shops', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}}], 'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da', 'name': 'Starbucks'}], 'closed_bucket': 'VeryLikelyOpen', 'distance': 2074, 'geocodes': {'drop_off': {'latitude': 40.779951, 'longitude': -73.959474}, 'main': {'latitude': 40.780038, 'longitude': -73.959593}, 'roof': {'latitude': 40.780038, 'longitude': -73.959593}}, 'link': '/v3/places/57fd4579498e7ebff233ef5a', 'location': {'address': '1142 Madison Ave', 'census_block': '360610150012001', 'country': 'US', 'cross_street': 'at E 85th St', 'dma': 'New York', 'formatted_address': '1142 Madison Ave (at E 85th St), New York, NY 10028', 'locality': 'New York', 'postcode': '10028', 'region': 'NY'}, 'name': 'Starbucks', 'related_places': {}, 'timezone': 'America/New_York'}, {'fsq_id': '5c1

In [64]:
basketball_stadiums_NYC = foursquare_places ("Basketball stadium",new_york_loc,"NYCbaskStadium")

[{'fsq_id': '4ce1ccc5c4f6a35de151dd6c', 'categories': [{'id': 12019, 'name': 'College Basketball Court', 'short_name': 'Basketball', 'plural_name': 'College Basketball Courts', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_basketball_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 1767, 'geocodes': {'main': {'latitude': 40.809935, 'longitude': -73.962017}, 'roof': {'latitude': 40.809935, 'longitude': -73.962017}}, 'link': '/v3/places/4ce1ccc5c4f6a35de151dd6c', 'location': {'address': '3030 Broadway', 'census_block': '360610203002000', 'country': 'US', 'cross_street': 'W 119th St & Broadway', 'dma': 'New York', 'formatted_address': '3030 Broadway (W 119th St & Broadway), New York, NY 10027', 'locality': 'New York', 'postcode': '10027', 'region': 'NY'}, 'name': 'Levien Gym - Columbia University', 'related_places': {'parent': {'fsq_id': '4227a500f964a520c51f1fe3', 'categories': [{'id': 12013, 'name': 'College and

In [83]:
schools_NYC = foursquare_places ("schools",new_york_loc,"NYCschools")

[{'fsq_id': '4c006f8f34ccc92861a1e1cd', 'categories': [{'id': 12110, 'name': 'Synagogue', 'short_name': 'Synagogue', 'plural_name': 'Synagogues', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/religious_synagogue_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 31784, 'geocodes': {'drop_off': {'latitude': 40.637738, 'longitude': -73.635609}, 'main': {'latitude': 40.638003, 'longitude': -73.635787}, 'roof': {'latitude': 40.638003, 'longitude': -73.635787}}, 'link': '/v3/places/4c006f8f34ccc92861a1e1cd', 'location': {'address': '3050 Oceanside Rd', 'census_block': '360594132005007', 'country': 'US', 'cross_street': '', 'dma': 'New York', 'formatted_address': '3050 Oceanside Rd, Oceanside, NY 11572', 'locality': 'Oceanside', 'postcode': '11572', 'region': 'NY'}, 'name': 'Temple Avodah', 'related_places': {}, 'timezone': 'America/New_York'}, {'fsq_id': '4af59efef964a5206ffa21e3', 'categories': [{'id': 12013, 'name': 'College and Univ

In [66]:
airport_NYC = foursquare_places ("airport",new_york_loc,"NYCairport")

[{'fsq_id': '7af888c27c5e4a0520f02898', 'categories': [{'id': 19020, 'name': 'Parking', 'short_name': 'Parking', 'plural_name': 'Parking', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/parking_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyClosed', 'distance': 1358, 'geocodes': {'drop_off': {'latitude': 40.786153, 'longitude': -73.955655}, 'main': {'latitude': 40.785917, 'longitude': -73.955755}, 'roof': {'latitude': 40.785917, 'longitude': -73.955755}}, 'link': '/v3/places/7af888c27c5e4a0520f02898', 'location': {'address': '22 E 94th St', 'address_extended': '# 61', 'census_block': '360610160013000', 'country': 'US', 'dma': 'New York', 'formatted_address': '22 E 94th St, New York, NY 10128', 'locality': 'New York', 'postcode': '10128', 'region': 'NY'}, 'name': 'Dollar Airport Parking', 'related_places': {}}, {'fsq_id': '56a35612cfc5487220908eaf', 'categories': [{'id': 19049, 'name': 'Taxi Stand', 'short_name': 'Taxi Stand', 'plural_name': 'Taxi Sta

In [67]:
train_stations_NYC = foursquare_places ("train station",new_york_loc,"NYCtrain_stations")

[{'fsq_id': '49de0156f964a52021601fe3', 'categories': [{'id': 19047, 'name': 'Rail Station', 'short_name': 'Rail Station', 'plural_name': 'Rail Stations', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 1262, 'geocodes': {'main': {'latitude': 40.805361, 'longitude': -73.939189}, 'roof': {'latitude': 40.805361, 'longitude': -73.939189}}, 'link': '/v3/places/49de0156f964a52021601fe3', 'location': {'address': '101 E 125th St', 'census_block': '360610198001000', 'country': 'US', 'cross_street': 'at Park Ave', 'dma': 'New York', 'formatted_address': '101 E 125th St (at Park Ave), New York, NY 10035', 'locality': 'New York', 'postcode': '10035', 'region': 'NY'}, 'name': 'Harlem–125th Street', 'related_places': {'children': [{'fsq_id': '5f8c55d1eb8bb56d90ef4c40', 'categories': [{'id': 19047, 'name': 'Rail Station', 'short_name': 'Rail Station', 'plural_name': 'Rail Stations', 'i

In [84]:
pet_grooming_NYC = foursquare_places ("pet grooming",new_york_loc,"NYCpet_grooming")

[{'fsq_id': '4b7961b5f964a520bef62ee3', 'categories': [{'id': 11134, 'name': 'Pet Grooming Service', 'short_name': 'Pet Grooming Service', 'plural_name': 'Pet Grooming Services', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/dogrun_', 'suffix': '.png'}}, {'id': 17110, 'name': 'Pet Supplies Store', 'short_name': 'Pet Supplies Store', 'plural_name': 'Pet Supplies Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 9593, 'geocodes': {'main': {'latitude': 40.711202, 'longitude': -73.959379}, 'roof': {'latitude': 40.711202, 'longitude': -73.959379}}, 'link': '/v3/places/4b7961b5f964a520bef62ee3', 'location': {'address': '235 Roebling St', 'census_block': '360470523002001', 'country': 'US', 'cross_street': 'at S 3rd St', 'dma': 'New York', 'formatted_address': '235 Roebling St (at S 3rd St), Brooklyn, NY 11211', 'locality': 'Brooklyn', 'postcode': '

In [69]:
dog_parks_NYC = foursquare_places ("dog park",new_york_loc,"NYCdog_parks")

[{'fsq_id': '4c1e5306b9f876b0fbe67e46', 'categories': [{'id': 16033, 'name': 'Dog Park', 'short_name': 'Dog Park', 'plural_name': 'Dog Parks', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/dogrun_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 812, 'geocodes': {'main': {'latitude': 40.802553, 'longitude': -73.943279}, 'roof': {'latitude': 40.802553, 'longitude': -73.943279}}, 'link': '/v3/places/4c1e5306b9f876b0fbe67e46', 'location': {'address': 'E 120th St at Madison Ave', 'census_block': '360610198001009', 'country': 'US', 'cross_street': 'at E 120th St', 'dma': 'New York', 'formatted_address': 'E 120th St at Madison Ave (at E 120th St), New York, NY 10035', 'locality': 'New York', 'postcode': '10035', 'region': 'NY'}, 'name': 'Marcus Garvey Park - Dog Run', 'related_places': {'parent': {'fsq_id': '4a9326bff964a520f81e20e3', 'categories': [{'id': 16037, 'name': 'Playground', 'short_name': 'Playground', 'plural_name': 'P

In [106]:
bars_clubs_NYC = foursquare_places ("night club",new_york_loc,"NYCbars_clubs")

[{'fsq_id': '4a46e401f964a520a0a91fe3', 'categories': [{'id': 10032, 'name': 'Night Club', 'short_name': 'Night Club', 'plural_name': 'Night Clubs', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'short_name': 'Restaurant', 'plural_name': 'Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 10287, 'geocodes': {'main': {'latitude': 40.706764, 'longitude': -73.923601}, 'roof': {'latitude': 40.706764, 'longitude': -73.923601}}, 'link': '/v3/places/4a46e401f964a520a0a91fe3', 'location': {'address': '2 Wyckoff Ave', 'census_block': '360470447002001', 'country': 'US', 'cross_street': 'at Jefferson St', 'dma': 'New York', 'formatted_address': '2 Wyckoff Ave (at Jefferson St), Brooklyn, NY 11237', 'locality': 'Brooklyn', 'postcode': '11237', 'region': 'NY'}, 'name': 'House of Yes', 'rela

In [86]:
vegan_vegetarian_NYC = foursquare_places ("vegan restaurant",new_york_loc,"NYCvegan_vegetarian")

[{'fsq_id': '503fb6d4ebca66a84f029bd8', 'categories': [{'id': 13002, 'name': 'Bakery', 'short_name': 'Bakery', 'plural_name': 'Bakeries', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'short_name': 'Restaurant', 'plural_name': 'Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 7613, 'geocodes': {'main': {'latitude': 40.736592, 'longitude': -73.99193}, 'roof': {'latitude': 40.736592, 'longitude': -73.99193}}, 'link': '/v3/places/503fb6d4ebca66a84f029bd8', 'location': {'address': '18 E 16th St', 'census_block': '360610052002005', 'country': 'US', 'cross_street': 'btwn Union Sq W & 5th Ave', 'dma': 'New York', 'formatted_address': '18 E 16th St (btwn Union Sq W & 5th Ave), New York, NY 10003', 'locality': 'New York', 'postcode': '10003', 'region': 'NY'}, 'name': 'Breads Bakery', 'related_pla

    SEATTLE

In [87]:
starbucks_ST = foursquare_places ("Starbucks",seattle_loc,"STstarbucks")

[{'fsq_id': '4b670e59f964a52066372be3', 'categories': [{'id': 13035, 'name': 'Coffee Shop', 'short_name': 'Coffee Shop', 'plural_name': 'Coffee Shops', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}}], 'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da', 'name': 'Starbucks'}], 'closed_bucket': 'VeryLikelyOpen', 'distance': 1669, 'geocodes': {'main': {'latitude': 47.614074, 'longitude': -122.32826}, 'roof': {'latitude': 47.614074, 'longitude': -122.32826}}, 'link': '/v3/places/4b670e59f964a52066372be3', 'location': {'address': '1124 Pike St', 'census_block': '530330083002004', 'country': 'US', 'cross_street': 'at Minor Ave', 'dma': 'Seattle-Tacoma', 'formatted_address': '1124 Pike St (at Minor Ave), Seattle, WA 98101', 'locality': 'Seattle', 'postcode': '98101', 'region': 'WA'}, 'name': 'Starbucks', 'related_places': {'children': [{'fsq_id': '5a020ad26fd62646ba6f6f90', 'categories': [{'id': 13002, 'name': 'Bakery', 'short_name': 'Bak

In [73]:
basketball_stadiums_ST = foursquare_places ("Basketball stadium",seattle_loc,"STbaskStadium")

[{'fsq_id': '5aecfcb56a5950002c5de373', 'categories': [{'id': 18048, 'name': 'Recreation Center', 'short_name': 'Recreation Center', 'plural_name': 'Recreation Centers', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 800, 'geocodes': {'main': {'latitude': 47.607231, 'longitude': -122.313405}, 'roof': {'latitude': 47.607231, 'longitude': -122.313405}}, 'link': '/v3/places/5aecfcb56a5950002c5de373', 'location': {'address': '550 14th Ave E', 'census_block': '530330086001012', 'country': 'US', 'cross_street': 'Cherry Street', 'dma': 'Seattle-Tacoma', 'formatted_address': '550 14th Ave E (Cherry Street), Seattle, WA 98112', 'locality': 'Seattle', 'postcode': '98112', 'region': 'WA'}, 'name': 'Redhawk Center', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4b15508bf964a520c1b023e3', 'categories': [{'id': 12019, 'name': 'College Basketball Court', 'shor

In [88]:
schools_ST = foursquare_places ("schools",seattle_loc,"STschools")

[{'fsq_id': '4b98214cf964a520822e35e3', 'categories': [{'id': 12059, 'name': 'High School', 'short_name': 'High School', 'plural_name': 'High Schools', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 9491, 'geocodes': {'main': {'latitude': 47.676926, 'longitude': -122.37456}, 'roof': {'latitude': 47.676926, 'longitude': -122.37456}}, 'link': '/v3/places/4b98214cf964a520822e35e3', 'location': {'address': '1418 NW 65th St', 'census_block': '530330033023005', 'country': 'US', 'dma': 'Seattle-Tacoma', 'formatted_address': '1418 NW 65th St, Seattle, WA 98117', 'locality': 'Seattle', 'postcode': '98117', 'region': 'WA'}, 'name': 'Ballard High School', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4aeb5318f964a520eec021e3', 'categories': [{'id': 12059, 'name': 'High School', 'short_name': 'High School', 'plural_name': 'High Schools', 'icon': {'prefix': 'https

In [75]:
airport_ST = foursquare_places ("airport",seattle_loc,"STairport")

[{'fsq_id': '4b3bfb6df964a520c17f25e3', 'categories': [{'id': 19047, 'name': 'Rail Station', 'short_name': 'Rail Station', 'plural_name': 'Rail Stations', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 976, 'geocodes': {'main': {'latitude': 47.602564, 'longitude': -122.331498}, 'roof': {'latitude': 47.602564, 'longitude': -122.331498}}, 'link': '/v3/places/4b3bfb6df964a520c17f25e3', 'location': {'address': '3RD Ave', 'census_block': '530330081022037', 'country': 'US', 'cross_street': 'at James St', 'dma': 'Seattle-Tacoma', 'formatted_address': '3RD Ave (at James St), Seattle, WA 98104', 'locality': 'Seattle', 'postcode': '98104', 'region': 'WA'}, 'name': 'Pioneer Square Link Station', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '510d23c0e889925f48ff1004', 'categories': [{'id': 19000, 'name': 'Travel and Transportation', 'short_name': 'Travel and Tran

In [76]:
train_stations_ST = foursquare_places ("train station",seattle_loc,"STtrain_stations")

[{'fsq_id': '148dad5889f64aa1748126fd', 'categories': [{'id': 19047, 'name': 'Rail Station', 'short_name': 'Rail Station', 'plural_name': 'Rail Stations', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'Unsure', 'distance': 779, 'geocodes': {'main': {'latitude': 47.59816, 'longitude': -122.327909}, 'roof': {'latitude': 47.59816, 'longitude': -122.327909}}, 'link': '/v3/places/148dad5889f64aa1748126fd', 'location': {'address': '505 5th Ave S', 'address_extended': 'Ste 630', 'census_block': '530330093002000', 'country': 'US', 'dma': 'Seattle-Tacoma', 'formatted_address': '505 5th Ave S, Seattle, WA 98104', 'locality': 'Seattle', 'postcode': '98104', 'region': 'WA'}, 'name': 'Talgo', 'related_places': {}}, {'fsq_id': '5a30481b6dcf044402d80418', 'categories': [{'id': 19047, 'name': 'Rail Station', 'short_name': 'Rail Station', 'plural_name': 'Rail Stations', 'icon': {'prefix': 'https://ss3.4sqi.net/img/c

In [89]:
pet_grooming_ST = foursquare_places ("pet grooming",seattle_loc, "STpet_grooming")

[{'fsq_id': '4f8f5124003960ba6c1c389b', 'categories': [{'id': 17110, 'name': 'Pet Supplies Store', 'short_name': 'Pet Supplies Store', 'plural_name': 'Pet Supplies Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyClosed', 'distance': 1250, 'geocodes': {'main': {'latitude': 47.600787, 'longitude': -122.335039}, 'roof': {'latitude': 47.600787, 'longitude': -122.335039}}, 'link': '/v3/places/4f8f5124003960ba6c1c389b', 'location': {'address': '87 1/2 S Washington St', 'census_block': '530330092002018', 'country': 'US', 'cross_street': 'btwn 1st Ave S & Alaskan Way S', 'dma': 'Seattle-Tacoma', 'formatted_address': '87 1/2 S Washington St (btwn 1st Ave S & Alaskan Way S), Seattle, WA 98104', 'locality': 'Seattle', 'postcode': '98104', 'region': 'WA'}, 'name': 'Pioneer Pet Feed & Supply', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4aa317bef964a520494320e3', 'categories':

In [78]:
dog_parks_ST = foursquare_places ("dog park",seattle_loc,"STdog_parks")

[{'fsq_id': '4a7f26dff964a5200df31fe3', 'categories': [{'id': 16032, 'name': 'Park', 'short_name': 'Park', 'plural_name': 'Parks', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 874, 'geocodes': {'main': {'latitude': 47.593025, 'longitude': -122.318285}}, 'link': '/v3/places/4a7f26dff964a5200df31fe3', 'location': {'address': '1008 12TH Ave S', 'census_block': '530330093001031', 'country': 'US', 'cross_street': 'at S. Judkins St.', 'dma': 'Seattle-Tacoma', 'formatted_address': '1008 12TH Ave S (at S. Judkins St.), Seattle, WA 98144', 'locality': 'Seattle', 'postcode': '98144', 'region': 'WA'}, 'name': 'Dr Jose Rizal Park', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '550e0142498e3245bcc97959', 'categories': [{'id': 16033, 'name': 'Dog Park', 'short_name': 'Dog Park', 'plural_name': 'Dog Parks', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v

In [107]:
bars_clubs_ST = foursquare_places ("night club",seattle_loc,"STbars_clubs")

[{'fsq_id': '40c8f600f964a520fd001fe3', 'categories': [{'id': 10037, 'name': 'Concert Hall', 'short_name': 'Concert Hall', 'plural_name': 'Concert Halls', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_', 'suffix': '.png'}}, {'id': 10041, 'name': 'Rock Club', 'short_name': 'Rock Club', 'plural_name': 'Rock Clubs', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_rockclub_', 'suffix': '.png'}}, {'id': 13003, 'name': 'Bar', 'short_name': 'Bar', 'plural_name': 'Bars', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 1796, 'geocodes': {'main': {'latitude': 47.608563, 'longitude': -122.339271}, 'roof': {'latitude': 47.608563, 'longitude': -122.339271}}, 'link': '/v3/places/40c8f600f964a520fd001fe3', 'location': {'address': '1426 1st Ave', 'census_block': '530330081012003', 'country': 'US', 'cross_street': '

In [91]:
vegan_vegetarian_ST = foursquare_places ("vegan restaurant",seattle_loc,"STvegan_vegetarian")

[{'fsq_id': '49d546d0f964a520845c1fe3', 'categories': [{'id': 17065, 'name': 'Farmers Market', 'short_name': 'Farmers Market', 'plural_name': 'Farmers Markets', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_farmersmarket_', 'suffix': '.png'}}, {'id': 17067, 'name': 'Fruit and Vegetable Store', 'short_name': 'Fruit and Vegetable Store', 'plural_name': 'Fruit and Vegetable Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}}, {'id': 17069, 'name': 'Grocery Store', 'short_name': 'Grocery Store', 'plural_name': 'Grocery Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 8924, 'geocodes': {'main': {'latitude': 47.667235, 'longitude': -122.383831}, 'roof': {'latitude': 47.667235, 'longitude': -122.383831}}, 'link': '/v3/places/49d546d0f964a520845c1fe3', 'location': {'address': '5330 Ballard Ave NW'